In [ ]:
#The workflow for the chart is like this:
# First, edit the data in the main xlsx.
# Then, update the local xlsx, which should have a data query in it to the relevant table
# Then, save this as a csv (probably possible to eliminate this step)
# Then, run this script to create the graph. Edit parameters as needed

In [ ]:

# pio.renderers.default = "iframe"
# for some reason the native renderer doesn't work in anaconda/jupyter

#Use this to write an html if needed
# import plotly.io as pio
# pio.write_html(fig, file=’bubblechart.html’, auto_open=True)

In [2]:
import plotly.io as pio
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio

# For trendline
import statsmodels.api as sm # method 1
import numpy as np # method 2

username = 'jaemikew' 
api_key = 'jjXZuDDrkKTbsefPaohg'

dataframe_bubble = pd.read_csv("datasets/graph_table.csv", sep=";", decimal=",")
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

pio.renderers.default = 'iframe'

In [9]:
def generate_bubbles(w, h, size, y, yintercept, sr, yrb, yrt, tty, ttx, logy, wd, tl, dt):
    fig = px.scatter(dataframe_bubble, 
                x="Intergenerational Solidarity Index", 
                y= y, 
                size="Population", 
                color="Region",
                hover_name="Country", 
                size_max = size,
                log_y= logy,
                )
    
    # Trendline method 1 - OLS
    # y_trendline = sm.OLS(dataframe_bubble[y],sm.add_constant(dataframe_bubble["Intergenerational Solidarity Index"])).fit().fittedvalues
    # Trendline method 2
    #z = np.polyfit(dataframe_bubble["Intergenerational Solidarity Index"], dataframe_bubble["v2x_libdem"], 1)
    #p = np.poly1d(z)
    #y_trendline = p(dataframe_bubble["Intergenerational Solidarity Index"])
    #fig.add_traces(go.Scatter(x=dataframe_bubble["Intergenerational Solidarity Index"], y=y_trendline,
    #                         mode = 'lines',
    #                         marker_color='black',
    #                         name='trend all')
    #)
    
    # Trendline method 1 - LOWESS
    lowess = sm.nonparametric.lowess(dataframe_bubble["v2x_libdem"].values,dataframe_bubble["Intergenerational Solidarity Index"].values)
    lowess_x = list(zip(*lowess))[0]
    lowess_y = list(zip(*lowess))[1]
    fig.add_traces(go.Scatter(x=lowess_x, y=lowess_y,
                             mode = 'lines',
                             marker_color='black',
                             name='trend all')
    )
    
    fig.add_shape(
        # Line reference to the axes
            type="line",
            xref="x",
            yref="y",
            x0=-20,
            y0=yintercept,
            x1=120,
            y1=yintercept,
            line=dict(
                color="#456987",
                width=wd,
            ),
            layer='below'
        )
    fig.add_shape(
        # Line reference to the axes
            type="line",
            xref="x",
            yref="y",
            x0=50,
            y0=-20,
            x1=50,
            y1=120,
            line=dict(
                color="#456987",
                width=wd,
            ),
            layer='below'
        )   
    fig.update_traces(line_color='#456987')
    fig.update_layout(showlegend=False,
                    width = w,
                    height = h,
                    margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=0
                    ),
                    autosize=False,
                    plot_bgcolor="#eeeeee",
                    yaxis = dict(
                    scaleanchor = "x",
                    scaleratio = sr,) 
                    )
    fig.update_xaxes(title_text= ttx,
                    showgrid=True,
                    zeroline=False,
                    range=[0, 100],
                    showticklabels=True,
                    )
    fig.update_yaxes(title_text= tty,
                    showgrid=True,
                    zeroline=False,
                    range = [yrb, yrt],
                    showticklabels=True,
                    dtick=dt,
                    )
    return fig

configurations = [
    {"name": "VDemFullScreen", "w": 500, "h": 500, "size": 60, "dt":0.2, "tl":"lowess", "y": "v2x_libdem","yintercept":0.45, "sr": 111, "yrb":0, "yrt":0.9, "tty":"VDem Liberal Democracy", "ttx":"Intergenerational Solidarity Index","logy":False, "wd":1},
    {"name": "VDemMobile", "w": 250, "h": 250, "size": 20, "dt":0.2, "tl":"lowess", "y": "v2x_libdem", "yintercept":0.45, "sr": 111,"yrb":0, "yrt":0.9, "tty":"VDem Lib. Democracy", "ttx": "ISI","logy":False, "wd":1},
   
    {"name": "FSIFullScreen", "w": 500, "h": 500, "size": 60, "dt":20, "tl":"lowess", "y": "FSI","yintercept":50, "sr": 1, "yrb":0,"yrt":100, "tty":"Fragile States Index", "ttx": "Intergenerational Solidarity Index","logy":False, "wd":0},
    {"name": "FSIMobile", "w": 250, "h": 250, "size": 20, "dt":20, "tl":"lowess", "y": "FSI", "yintercept":50, "sr": 1,"yrb":0, "yrt":100,"tty":"State Fragility", "ttx": "ISI", "logy":False, "wd":0},
   
    {"name": "DIFullScreen", "w": 500, "h": 500, "size": 60, "dt":1, "tl":"none", "y": "AdjDi","yintercept":50, "sr": 0.1, "yrb":-3,"yrt":2, "tty":"Decentralisation Index (Adjusted)", "ttx": "Intergenerational Solidarity Index","logy":True, "wd":0},
    {"name": "DIMobile", "w": 250, "h": 250, "size": 20, "dt":1, "tl":"none", "y": "AdjDi", "yintercept":50, "sr": 0.1, "yrb":-3, "yrt":2,"tty":"Decentralisation", "ttx": "ISI", "logy":True, "wd":0},
]

for item in configurations:
    fig = generate_bubbles(w=item["w"], 
                           h=item["h"], 
                           size=item["size"], 
                           y=item["y"], 
                           yintercept=item["yintercept"], 
                           sr=item["sr"], 
                           yrb=item["yrb"], 
                           yrt=item["yrt"], 
                           tty=item["tty"], 
                           ttx=item["ttx"], 
                           logy=item["logy"], 
                           wd=item["wd"],
                           tl=item["tl"],
                           dt=item["dt"]
                          )
    fig.show()
    py.plot(fig, filename = item["name"], auto_open=False)